In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from mezo.currency_utils import format_musd_currency_columns, get_token_price, format_currency_columns
from mezo.datetime_utils import format_datetimes
from mezo.data_utils import add_rolling_values, add_pct_change_columns, add_cumulative_columns
from mezo.clients import SubgraphClient, Web3Client
from mezo.queries import MUSDQueries
load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

In [ ]:
# import raw data
# raw_loans = SubgraphClient.get_subgraph_data(SubgraphClient.MUSD_TROVE_MANAGER_SUBGRAPH, )
# raw_liquidations = get_liquidation_data()
# raw_troves_liquidated = get_trove_liquidated_data()

raw_loans = SubgraphClient.get_subgraph_data(
    SubgraphClient.BORROWER_OPS_SUBGRAPH,
    MUSDQueries.GET_LOANS,
    'troveUpdateds'
)

raw_liquidations = SubgraphClient.get_subgraph_data(
    SubgraphClient.MUSD_TROVE_MANAGER_SUBGRAPH,
    MUSDQueries.GET_MUSD_LIQUIDATIONS,
    'liquidations'
)

raw_troves_liquidated = SubgraphClient.get_subgraph_data(
    SubgraphClient.MUSD_TROVE_MANAGER_SUBGRAPH,
    MUSDQueries.GET_LIQUIDATED_TROVES,
    'troveLiquidateds'
)

In [2]:
# helpers
def clean_loan_data(raw, sort_col, date_cols, currency_cols):
    df = raw.copy().sort_values(by=sort_col, ascending=False)
    df = format_datetimes(df, date_cols)
    df = format_musd_currency_columns(df, currency_cols)
    df['count'] = 1
    df['id'] = range(1, len(df) + 1)

    return df

def find_coll_ratio(df, token_id):
    """Computes the collateralization ratio"""
    usd = get_token_price(token_id)
    df['coll_usd'] = df['coll'] * usd
    df['coll_ratio'] = (df['coll_usd']/df['principal'] ).fillna(0)

    return df

def get_loans_subset(df, operation: int, equals):
    """Create a df with only new, adjusted, or closed loans
    0 = opened, 1 = closed, 2 = adjusted
    note: operation = 2 also includes liquidated loans, so we have to remove those manually
    """
    df['operation'] = df['operation'].astype(int)
    if equals is True:
        adjusted = df.loc[df['operation'] == operation]
    elif equals is False:
        adjusted = df.loc[df['operation'] != operation]

    return adjusted

def process_liquidation_data(liquidations, troves_liquidated):
    # Merge raw liquidation data from two queries
    liquidation_df_merged = pd.merge(
        liquidations, 
        troves_liquidated, 
        how='left', 
        on='transactionHash_'
    )

    liquidation_df_merged = liquidation_df_merged[
        ['timestamp__x', 
        'liquidatedPrincipal', 
        'liquidatedInterest', 
        'liquidatedColl', 
        'borrower',
        'transactionHash_',
        'count_x'
        ]
    ]

    liquidations_df_final = liquidation_df_merged.rename(
        columns = {
            'timestamp__x': 'timestamp_', 
            'liquidatedPrincipal': 'principal', 
            'liquidatedInterest': 'interest',
            'liquidatedColl': 'coll',
            'count_x': 'count'
        }
    )

    liquidations_final = liquidations_df_final.copy()
    liquidations_final['coll'] = liquidations_final['coll'].astype(float)

    return liquidations_final

In [ ]:
# clean raw data
loans = clean_loan_data(
    raw_loans, 
    sort_col='timestamp_', 
    date_cols=['timestamp_'], 
    currency_cols=['principal', 'coll', 'stake', 'interest']
)

loans = find_coll_ratio(loans, 'bitcoin')

liquidations = clean_loan_data(
    raw_liquidations,
    sort_col='timestamp_',
    date_cols=['timestamp_'],
    currency_cols=['liquidatedPrincipal', 'liquidatedInterest', 'liquidatedColl']
)

troves_liquidated = clean_loan_data(
    raw_troves_liquidated,
    sort_col='timestamp_',
    date_cols=['timestamp_'],
    currency_cols=['debt', 'coll']
)

# Create df for liquidated loans
liquidations_final = process_liquidation_data(liquidations, troves_liquidated)

# Create df's for new loans, closed loans, and adjusted loans and upload to BigQuery
new_loans = get_loans_subset(loans, 0, True)
closed_loans = get_loans_subset(loans, 1, True)
adjusted_loans = get_loans_subset(loans, 2, True) # Only adjusted loans (incl multiple adjustments from a single user)

## Remove liquidations from adjusted loans
liquidated_borrowers = liquidations_final['borrower'].unique()
adjusted_loans = adjusted_loans[~adjusted_loans['borrower'].isin(liquidated_borrowers)]

##################################

# Get latest loans
latest_loans = loans.drop_duplicates(subset='borrower', keep='first')

# Create df with only open loans
latest_open_loans = get_loans_subset(latest_loans, 1, False)

# Remove liquidated loans from list of latest loans w/o closed loans
latest_open_loans = latest_open_loans[~latest_open_loans['borrower'].isin(liquidated_borrowers)]

##################################

# Break down adjusted loan types for analysis
adjusted_loans = adjusted_loans.sort_values(by=['borrower', 'timestamp_'])
first_tx = adjusted_loans.groupby('borrower').first().reset_index()

adjusted_loans_merged = adjusted_loans.merge(
    first_tx[['borrower', 'principal', 'coll']], 
    on='borrower', 
    suffixes=('', '_initial')
)

## Loan increases
increased_loans = adjusted_loans_merged[adjusted_loans_merged['principal'] 
                                        > adjusted_loans_merged['principal_initial']].copy()
increased_loans['type'] = 1

## Collateral changes
coll_increased = adjusted_loans_merged[adjusted_loans_merged['coll'] 
                                       > adjusted_loans_merged['coll_initial']].copy()
coll_increased['type'] = 2

coll_decreased = adjusted_loans_merged[adjusted_loans_merged['coll'] 
                                       < adjusted_loans_merged['coll_initial']].copy()
coll_decreased['type'] = 3

## MUSD Repayments
principal_decreased = adjusted_loans_merged[adjusted_loans_merged['principal'] 
                                            < adjusted_loans_merged['principal_initial']].copy()
principal_decreased['type'] = 4

## Create final_adjusted_loans dataframe with type column
final_adjusted_loans = pd.concat([
    increased_loans,
    coll_increased, 
    coll_decreased,
    principal_decreased
], ignore_index=True)

In [ ]:
loans

In [ ]:
new_loans

In [ ]:
GET_REDEMPTIONS = """
query getRedemptions($skip: Int!) {
    redemptions(
    first: 1000
    orderBy: timestamp_
    orderDirection: desc
    skip: $skip
  ) {
    timestamp_
    actualAmount
    attemptedAmount
    collateralFee
    collateralSent
    transactionHash_
    block_number
  }
}
"""

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

In [ ]:
def get_redemptions():
    """
    Get redemption data for troves from the MUSD Trove Manager subgraph
    """
    musd = SubgraphClient(
        url=SubgraphClient.MUSD_TROVE_MANAGER_SUBGRAPH, 
        headers=SubgraphClient.SUBGRAPH_HEADERS
    )
    
    print("🔍 Trying redemptions query...")
    try:
        redemptions_data = musd.fetch_subgraph_data(
            GET_REDEMPTIONS, 
            'redemptions'
        )
    
        if redemptions_data:
            redemptions_df = pd.DataFrame(redemptions_data)
            print(f"✅ Found {len(redemptions_df)} redemption records")
            
            return redemptions_df
        else:
            print("⚠️ redemptions query returned no data")
    except Exception as e:
        print(f"❌ redemptions query failed: {e}")

In [ ]:
raw_redemptions = get_redemptions()

In [ ]:
raw_redemptions

In [ ]:
redemptions_clean = clean_loan_data(
    raw_redemptions,
    sort_col='timestamp_',
    date_cols=['timestamp_'],
    currency_cols=['actualAmount', 'attemptedAmount', 'collateralFee', 'collateralSent']
)
redemptions_clean

In [ ]:
refinanced_loans = get_loans_subset(loans, 3, True)
refinanced_loans

# Fees

In [ ]:
# Get borrowing rate

w3 = Web3Client('BorrowerOperations')
borrower_ops = w3.load_contract()
borrowing_rate = borrower_ops.functions.borrowingRate().call()

In [ ]:
# Get issuance fee

borrowing_rate = (borrowing_rate/1e18)
new_loans['issuance_fee'] = new_loans['principal'] * borrowing_rate

In [ ]:
# Get gas comp

trove = Web3Client('troveManager')
troves = trove.load_contract()
gas_comp = troves.functions.MUSD_GAS_COMPENSATION().call()
gas_comp = gas_comp / 1e18

In [ ]:
trove = Web3Client('troveManager')
troves = trove.load_contract()

In [ ]:
# Get refinancing rate

refinance_fee_pctage = borrower_ops.functions.refinancingFeePercentage().call()
borrowing_fee_pctage = borrowing_rate * 100
refinance_fee = refinance_fee_pctage/100
refinance_rate = borrowing_fee_pctage * refinance_fee
final_refinance_rate = refinance_rate/100

In [ ]:
# Calculate refinance fees and add to df

refinanced_loans['fees'] = (refinanced_loans['principal'] + refinanced_loans['interest'] - gas_comp) * final_refinance_rate

In [ ]:
redemptions_clean

In [ ]:
GET_BORROW_FEES = """
query getBorrowFees ($skip: Int!) {
  borrowingFeePaids (
    orderBy: timestamp_
    orderDirection: desc
    first: 1000
    skip: $skip
  ){
    timestamp_
    fee
    borrower
    transactionHash_
  }
}"""

In [ ]:
# Fetch borrow fees from subgraph 

musd = SubgraphClient(
    url=SubgraphClient.BORROWER_OPS_SUBGRAPH, 
    headers= SubgraphClient.SUBGRAPH_HEADERS
)
fees =  musd.fetch_subgraph_data(GET_BORROW_FEES, 'borrowingFeePaids')

if fees:
    fees = pd.DataFrame(fees)
    print(f"✅ Found {len(fees)} fee records")
else:
    print("⚠️ Query returned no data")

In [ ]:
test = pd.merge(raw_redemptions, fees, how='left')

In [ ]:
# MOVE THIS CODE CHUNK TO HEX
#  Add fees to loans df
########################

loan_fees = fees.copy()
loans_with_fees = pd.merge(loans, loan_fees, how='left', on='transactionHash_')

In [ ]:
# Create daily dataframe
daily_new_loans = new_loans.groupby(['timestamp_']).agg(
    loans_opened = ('count', 'sum'),
    borrowers = ('borrower', lambda x: x.nunique()),
    principal = ('principal', 'sum'),
    collateral = ('coll', 'sum'),
    interest = ('interest', 'sum')
).reset_index()

daily_closed_loans = closed_loans.groupby(['timestamp_']).agg(
    loans_closed = ('count', 'sum'),
    borrowers_who_closed = ('borrower', lambda x: x.nunique())
).reset_index()

daily_new_and_closed_loans = pd.merge(daily_new_loans, daily_closed_loans, how = 'outer', on = 'timestamp_').fillna(0)
daily_new_and_closed_loans[['loans_opened', 'borrowers', 'loans_closed', 'borrowers_who_closed']] = daily_new_and_closed_loans[['loans_opened', 'borrowers', 'loans_closed', 'borrowers_who_closed']].astype('int')      
daily_adjusted_loans = adjusted_loans.groupby(['timestamp_']).agg(
    loans_adjusted = ('count', 'sum'),
    borrowers_who_adjusted = ('borrower', lambda x: x.nunique())
).reset_index()

daily_loan_data = pd.merge(daily_new_and_closed_loans, daily_adjusted_loans, how='outer', on='timestamp_').fillna(0)
daily_loan_data[['loans_adjusted', 'borrowers_who_adjusted']] = daily_loan_data[['loans_adjusted', 'borrowers_who_adjusted']].astype(int)

daily_balances = latest_loans.groupby(['timestamp_']).agg(
    musd = ('principal', 'sum'),
    interest = ('interest', 'sum'),
    collateral = ('coll', 'sum')
).reset_index()

daily_balances = daily_balances.rename(
    columns={'musd': 'net_musd', 
             'interest': 'net_interest',
             'collateral': 'net_coll'}
)

daily_loans_merged = pd.merge(daily_loan_data, daily_balances, how='outer', on='timestamp_')

cols = {
    'timestamp_': 'date', 
    'principal': 'gross_musd', 
    'collateral': 'gross_coll', 
    'interest': 'gross_interest',
    'borrowers_who_closed': 'closers', 
    'borrowers_who_adjusted': 'adjusters'
}

daily_loans_merged = daily_loans_merged.rename(columns = cols)

daily_musd_final = add_rolling_values(daily_loans_merged, 30, ['net_musd', 'net_interest', 'net_coll']).fillna(0)
daily_musd_final_2 = add_cumulative_columns(daily_musd_final, ['net_musd', 'net_interest', 'net_coll'])
daily_musd_final_3 = add_pct_change_columns(daily_musd_final_2, ['net_musd', 'net_interest', 'net_coll'], 'daily').fillna(0)
final_daily_musd = daily_musd_final_3.replace([float('inf'), -float('inf')], 0)
final_daily_musd['date'] = pd.to_datetime(final_daily_musd['date']).dt.strftime('%Y-%m-%d')

In [ ]:
token = Web3Client('MUSD')
t = token.load_contract()
t.all_functions()
t.functions.mint(0).call()

In [ ]:
import requests
import pandas as pd
import time
from typing import Optional, Dict, Any
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def fetch_paginated_api_data(
    endpoint: str, 
    items_key: str = 'items',  # Key to extract items from response
    page_size: int = 1000,  # Items per page
    page_size_param: str = 'limit',  # Parameter name for page size
    max_retries: int = 3,
    timeout: int = 30,
    rate_limit_delay: float = 0.5,
    additional_params: Optional[Dict[str, Any]] = None
) -> pd.DataFrame:
    """
    Fetch data from the specified API endpoint with pagination.
    
    Args:
        endpoint: API endpoint path
        items_key: Key in response JSON that contains the items
        page_size: Number of items to request per page (default 1000)
        page_size_param: Parameter name for page size ('limit', 'per_page', etc.)
        max_retries: Maximum number of retries for failed requests
        timeout: Request timeout in seconds
        rate_limit_delay: Delay between requests to avoid rate limiting
        additional_params: Additional query parameters to include
    
    Returns:
        DataFrame with all fetched data
    """
    base_url = 'https://api.explorer.mezo.org/api/v2'
    url = f"{base_url}/{endpoint}"
    logger.info(f"Fetching data from: {url} with page size: {page_size}")
    
    all_data = []
    next_page_params = {}
    page_count = 0
    total_items = 0
    
    # Set up initial parameters
    if additional_params:
        next_page_params.update(additional_params)
    
    # Add page size parameter
    next_page_params[page_size_param] = page_size
    
    while True:
        retry_count = 0
        success = False
        
        # Retry logic
        while retry_count < max_retries and not success:
            try:
                logger.info(f"Fetching page {page_count + 1}, items so far: {total_items}")
                
                # Make request with current params
                response = requests.get(
                    url, 
                    params=next_page_params, 
                    timeout=timeout
                )
                
                # Check status code
                if response.status_code == 429:  # Rate limited
                    logger.warning("Rate limited, waiting longer...")
                    time.sleep(rate_limit_delay * 5)
                    retry_count += 1
                    continue
                elif response.status_code != 200:
                    logger.error(f"HTTP {response.status_code}: {response.text}")
                    raise Exception(f"Failed to fetch data: {response.status_code}")
                
                # Parse response
                data = response.json()
                items = data.get(items_key, [])
                
                if not items:
                    logger.info("No more items found, stopping pagination")
                    break
                
                # Convert to DataFrame and append
                df_chunk = pd.json_normalize(items)
                all_data.append(df_chunk)
                
                total_items += len(items)
                page_count += 1
                
                # Check for next page
                new_page_params = data.get("next_page_params")
                if not new_page_params:
                    logger.info("No next_page_params found, stopping pagination")
                    break
                
                # Update pagination params but keep page size
                next_page_params.update(new_page_params)
                next_page_params[page_size_param] = page_size  # Ensure page size is maintained
                
                success = True
                
                # Rate limiting delay
                time.sleep(rate_limit_delay)
                
            except requests.exceptions.Timeout:
                logger.warning(f"Timeout on retry {retry_count + 1}")
                retry_count += 1
                time.sleep(2 ** retry_count)  # Exponential backoff
                
            except requests.exceptions.RequestException as e:
                logger.error(f"Request failed on retry {retry_count + 1}: {e}")
                retry_count += 1
                time.sleep(2 ** retry_count)
                
            except Exception as e:
                logger.error(f"Unexpected error: {e}")
                raise
        
        if not success:
            raise Exception(f"Failed to fetch data after {max_retries} retries")
        
        # Break if we successfully processed a page with no next_page_params
        if not new_page_params:
            break
    
    logger.info(f"Completed! Fetched {total_items} items across {page_count} pages")
    
    # Combine all DataFrames
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        logger.info(f"Final DataFrame shape: {final_df.shape}")
        return final_df
    else:
        logger.warning("No data was fetched")
        return pd.DataFrame()


def fetch_single_page_data(endpoint: str, timeout: int = 30) -> pd.DataFrame:
    """
    Fetch data from a single page endpoint (non-paginated).
    """
    base_url = 'https://api.explorer.mezo.org/api/v2'
    url = f"{base_url}/{endpoint}"
    logger.info(f"Fetching single page from: {url}")
    
    try:
        response = requests.get(url, timeout=timeout)
        
        if response.status_code != 200:
            raise Exception(f"Failed to fetch data: {response.status_code} - {response.text}")
        
        data = response.json()
        return pd.json_normalize(data)
        
    except requests.exceptions.Timeout:
        raise Exception(f"Request timed out after {timeout} seconds")
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {e}")

In [6]:
mints = SubgraphClient.get_subgraph_data(SubgraphClient.MUSD_TOKEN_SUBGRAPH, MUSDQueries.GET_MUSD_MINTS, 'transfers')
burns = SubgraphClient.get_subgraph_data(SubgraphClient.MUSD_TOKEN_SUBGRAPH, MUSDQueries.GET_MUSD_BURNS, 'transfers')

🔍 Trying transfers query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
No more records found.
✅ Found 2467 transfers records
🔍 Trying transfers query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.
✅ Found 520 transfers records


In [7]:
burns.head()

,timestamp_,from,to,value,transactionHash_,block_number
0,1756912849,0x3eb418bdbe95b4b9cf465ecfbd8424685acd1bc1,0x0000000000000000000000000000000000000000,200000000000000000000,0x8a97436cd05ce0f705b41df26131b1579356473e2b98...,2910587
1,1756912849,0x6b4c85a028d1e88ed45f94ebc1f295760b4e7a4b,0x0000000000000000000000000000000000000000,14540034422404500818207,0x8a97436cd05ce0f705b41df26131b1579356473e2b98...,2910587
2,1756912705,0xaf4710372becbfbb2ce2719029b95d95d6b14ecd,0x0000000000000000000000000000000000000000,1348000000000000000000,0x33d8dd6df38334c0a6c8a4c78c7054840a1cd70b9290...,2910547
3,1756912278,0xaf4710372becbfbb2ce2719029b95d95d6b14ecd,0x0000000000000000000000000000000000000000,200000000000000000,0x77951ac240b7f7617c57c75efb545f90ce26b5af6935...,2910428
4,1756912051,0xaf4710372becbfbb2ce2719029b95d95d6b14ecd,0x0000000000000000000000000000000000000000,4950000000000000000,0x9501d8a998b993e738164ee2ff2374e4ce1fff7a303e...,2910365


In [11]:
burns_clean = format_datetimes(burns, ['timestamp_'])
burns_clean = format_musd_currency_columns(burns, ['value'])
burns_clean.head()

,timestamp_,from,to,value,transactionHash_,block_number
0,2025-09-03,0x3eb418bdbe95b4b9cf465ecfbd8424685acd1bc1,0x0000000000000000000000000000000000000000,200.000000,0x8a97436cd05ce0f705b41df26131b1579356473e2b98...,2910587
1,2025-09-03,0x6b4c85a028d1e88ed45f94ebc1f295760b4e7a4b,0x0000000000000000000000000000000000000000,14540.034422,0x8a97436cd05ce0f705b41df26131b1579356473e2b98...,2910587
2,2025-09-03,0xaf4710372becbfbb2ce2719029b95d95d6b14ecd,0x0000000000000000000000000000000000000000,1348.000000,0x33d8dd6df38334c0a6c8a4c78c7054840a1cd70b9290...,2910547
3,2025-09-03,0xaf4710372becbfbb2ce2719029b95d95d6b14ecd,0x0000000000000000000000000000000000000000,0.200000,0x77951ac240b7f7617c57c75efb545f90ce26b5af6935...,2910428
4,2025-09-03,0xaf4710372becbfbb2ce2719029b95d95d6b14ecd,0x0000000000000000000000000000000000000000,4.950000,0x9501d8a998b993e738164ee2ff2374e4ce1fff7a303e...,2910365


In [12]:
mints_clean = format_datetimes(mints, ['timestamp_'])
mints_clean = format_musd_currency_columns(mints_clean, ['value'])

In [13]:
mints_clean

,timestamp_,from,to,value,transactionHash_,block_number
0,2025-09-03,0x0000000000000000000000000000000000000000,0x391ecc7ffefc48cff41d0f2bb36e38b82180b993,1.913271,0x995165d2f8df8f8f6e2e96b483492d5fb624177b574b...,2912690
1,2025-09-03,0x0000000000000000000000000000000000000000,0x391ecc7ffefc48cff41d0f2bb36e38b82180b993,1.000000,0x0dbde803422dc2fca3fd9bfe288b3d4c46547de72ac5...,2912349
2,2025-09-03,0x0000000000000000000000000000000000000000,0xce96ac0be6a54d5c5bba49e90cc7f965581e5fde,1000.000000,0x0dbde803422dc2fca3fd9bfe288b3d4c46547de72ac5...,2912349
3,2025-09-03,0x0000000000000000000000000000000000000000,0x391ecc7ffefc48cff41d0f2bb36e38b82180b993,1.494390,0x0dbde803422dc2fca3fd9bfe288b3d4c46547de72ac5...,2912349
4,2025-09-03,0x0000000000000000000000000000000000000000,0x391ecc7ffefc48cff41d0f2bb36e38b82180b993,3.827326,0xd8b595bbe6b7a132fd187c7be1c7116040a07aa3720f...,2912083
...,...,...,...,...,...,...
2462,2025-05-22,0x0000000000000000000000000000000000000000,0x3eb418bdbe95b4b9cf465ecfbd8424685acd1bc1,200.000000,0xffda614cd610859c4f367e5fb3944368348a13f098e5...,420170
2463,2025-05-22,0x0000000000000000000000000000000000000000,0x391ecc7ffefc48cff41d0f2bb36e38b82180b993,0.003043,0xffda614cd610859c4f367e5fb3944368348a13f098e5...,420170
2464,2025-05-22,0x0000000000000000000000000000000000000000,0x3eb418bdbe95b4b9cf465ecfbd8424685acd1bc1,200.000000,0x443e5b3ed824450f27ae07519fe776bd9a4247194897...,418873
2465,2025-05-22,0x0000000000000000000000000000000000000000,0x123694886dbf5ac94dda07135349534536d14caf,1800.000000,0x443e5b3ed824450f27ae07519fe776bd9a4247194897...,418873


In [16]:
daily_mints = mints_clean.groupby(['timestamp_']).agg(
    mints = ('from', 'count'),
    minters = ('to', lambda x: x.nunique()),
    amt_minted = ('value', 'sum')
).reset_index()

daily_burns = burns_clean.groupby(['timestamp_']).agg(
    burns = ('from', 'count'),
    burners = ('from', lambda x: x.nunique()),
    amt_burned = ('value', 'sum')
).reset_index()


,timestamp_,mints,minters,amt_minted
0,2025-05-22,7,4,4203.803043
1,2025-05-23,37,7,16265.624404
2,2025-05-24,60,6,15446.396101
3,2025-05-25,31,6,8818.579127
4,2025-05-26,56,11,19241.051052
...,...,...,...,...
80,2025-08-30,15,2,10147.183015
81,2025-08-31,9,4,4091.224372
82,2025-09-01,8,3,11770.182484
83,2025-09-02,30,5,8039.382257


,timestamp_,burns,burners,amt_burned
0,2025-05-23,11,5,7008.014141
1,2025-05-24,24,5,15094.587703
2,2025-05-25,16,7,9007.122436
3,2025-05-26,23,10,19240.681413
4,2025-05-27,28,13,22134.221268
...,...,...,...,...
66,2025-08-30,12,4,34600.733734
67,2025-08-31,1,1,0.000000
68,2025-09-01,6,4,12515.438123
69,2025-09-02,19,8,65963.603957


In [31]:
daily_mints_and_burns = pd.merge(daily_mints, daily_burns, how='outer', on='timestamp_').fillna(0)
daily_mints_and_burns = add_rolling_values(daily_mints_and_burns, 7, cols=['amt_minted', 'amt_burned'])
daily_mints_and_burns = add_cumulative_columns(daily_mints_and_burns, cols=['mints', 'amt_minted', 'burns', 'amt_burned']).fillna(0)
daily_mints_and_burns

,timestamp_,mints,minters,amt_minted,burns,burners,amt_burned,rolling_amt_minted_7,rolling_amt_burned_7,cumulative_mints,cumulative_amt_minted,cumulative_burns,cumulative_amt_burned,cumulative_mints_growth,cumulative_amt_minted_growth,cumulative_burns_growth,cumulative_amt_burned_growth
0,2025-05-22,7.0,4.0,4203.803043,0.0,0.0,0.000000,4203.803043,0.000000,7.0,4.203803e+03,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1,2025-05-23,37.0,7.0,16265.624404,11.0,5.0,7008.014141,10234.713723,3504.007070,44.0,2.046943e+04,11.0,7.008014e+03,5.285714,3.869264,inf,inf
2,2025-05-24,60.0,6.0,15446.396101,24.0,5.0,15094.587703,11971.941183,7367.533948,104.0,3.591582e+04,35.0,2.210260e+04,1.363636,0.754608,2.181818,2.153904
3,2025-05-25,31.0,6.0,8818.579127,16.0,7.0,9007.122436,11183.600669,7777.431070,135.0,4.473440e+04,51.0,3.110972e+04,0.298077,0.245535,0.457143,0.407514
4,2025-05-26,56.0,11.0,19241.051052,23.0,10.0,19240.681413,12795.090745,10070.081138,191.0,6.397545e+04,74.0,5.035041e+04,0.414815,0.430118,0.450980,0.618478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,2025-08-30,15.0,2.0,10147.183015,12.0,4.0,34600.733734,150721.883011,13945.186033,2373.0,6.876896e+06,488.0,8.690386e+07,0.006361,0.001478,0.025210,0.000398
82,2025-08-31,9.0,4.0,4091.224372,1.0,1.0,0.000000,149208.867140,12008.544963,2382.0,6.880987e+06,489.0,8.690386e+07,0.003793,0.000595,0.002049,0.000000
83,2025-09-01,8.0,3.0,11770.182484,6.0,4.0,12515.438123,131350.204466,7996.879563,2390.0,6.892757e+06,495.0,8.691638e+07,0.003359,0.001711,0.012270,0.000144
84,2025-09-02,30.0,5.0,8039.382257,19.0,8.0,65963.603957,132483.794035,16496.413632,2420.0,6.900797e+06,514.0,8.698234e+07,0.012552,0.001166,0.038384,0.000759


In [ ]:
new_loans['issuance_fee'] = new_loans['principal'] * borrowing_rate

In [ ]:
import requests
import pandas as pd

trove_manager = "0x94afb503dbca74ac3e4929bacededfce19b93c193"
url = 'https://api.explorer.mezo.org/api/v2/addresses/0x94afb503dbca74ac3e4929baceedfce19b93c193/transactions'

# Create list to store all transaction data
transactions_list = []
next_page_params = None
page_count = 0

while True:
    page_count += 1
    print(f"Fetching page {page_count}...")

    # Build URL with pagination parameters
    if next_page_params:
        response = requests.get(url, params=next_page_params)
    else:
        response = requests.get(url)

    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        break
        
    data = response.json()
    transactions = data.get("items", [])

    if not transactions:
        print("No more transactions found.")
        break

    print(f"Found {len(transactions)} transactions on page {page_count}")

    for tx in transactions:
        tx_data = {
            'timestamp': tx.get('timestamp'),
            'method': tx.get('method'),
            'fee_value': int(tx.get('fee', {}).get('value', 0)) if tx.get('fee') else 0,
            'has_error': tx.get('has_error_in_internal_txs', False),
            'from_address': tx.get('from', {}).get('hash') if tx.get('from') else None,
            'to_address': tx.get('to', {}).get('hash') if tx.get('to') else None,
            'transactionHash_': tx.get('hash'),
            'block_number': tx.get('block'),
        }

        # Extract decoded input parameters if available
        if tx.get('decoded_input') and tx.get('decoded_input').get('parameters'):
            parameters = tx.get('decoded_input').get('parameters')
            if len(parameters) > 0:
                first_param = parameters[0]
                tx_data['param_0_name'] = first_param.get('name')
                tx_data['param_0_value'] = str(first_param.get('value'))
        
        transactions_list.append(tx_data)
      
    # Check for next page
    next_page_params = data.get("next_page_params")
    if not next_page_params:
        print("Reached last page.")
        break

# Create DataFrame
transactions_df = pd.DataFrame(transactions_list)

if len(transactions_df) > 0:
    # Display the DataFrame
    print(f"✅ Created DataFrame with {len(transactions_df)} total transactions")
    print(f"📊 Columns: {list(transactions_df.columns)}")
    print(f"🔧 Methods found: {transactions_df['method'].value_counts()}")
    print(f"📅 Date range: {transactions_df['timestamp'].min()} to {transactions_df['timestamp'].max()}")

    # Show sample data
    print(f"\n🔍 First 5 transactions:")
    print(transactions_df.head())

    # Show transaction fees by method
    print(f"\n💰 Average fees by method:")
    fee_by_method = transactions_df.groupby('method')['fee_value'].agg(['count', 'mean','sum']).round(0)
    print(fee_by_method)
else:
    print("❌ No transaction data was fetched. Check the API endpoint and parameters.")

In [ ]:
def get_token_price(token_id):
    url = 'https://api.coingecko.com/api/v3/simple/price'
    params = {'ids': token_id, 'vs_currencies': 'usd'}
    headers = {'x-cg-demo-api-key': COINGECKO_KEY}

    response = requests.get(url, params = params)

    data = response.json()
    price = data[token_id]['usd']
    
    return price

In [ ]:
def find_coll_ratio(df, token_id):
    """Computes the collateralization ratio"""
    usd = get_token_price(token_id)
    df['coll_usd'] = df['coll'] * usd
    df['coll_ratio'] = (df['coll_usd']/df['principal']).fillna(0)
    return df

In [ ]:
get_token_price('bitcoin')

In [ ]:
from decimal import Decimal
from mezo.clients import Web3Client

pcv_client = Web3Client('PCV')
print('OK')
pcv = pcv_client.load_contract()
print('OK')
pcv_loan_balance = pcv.functions.debtToPay().call()
print('OK')
pcv_loan_balance = float((Decimal(pcv_loan_balance) / Decimal("1e18")).normalize())